In [9]:
import os
import pandas as pd
from tqdm import tqdm
import re

In [92]:
# Open the dataset documents and store their data into a DataFrame
def load_himym_dataset():
    episodes_folder = os.path.join(os.getcwd(), "Download Dataset Script", "Dataset")
    dataframe_rows = []
    # Get number of documents and their names
    documents_n = len(os.listdir(episodes_folder))
    documents_names = os.listdir(episodes_folder)

    # Loop over documents
    for i in tqdm(range(documents_n)):
        filename = documents_names[i]
        # Open document
        file = open(os.path.join(episodes_folder, filename))
        episode_index = filename[:-4]
        # Loop over lines (= words)
        for line in file.readlines():
                dataframe_row = {
                    "episode": episode_index,
                    "line": line,
                }
                dataframe_rows.append(dataframe_row)
    # Build the dataframe from the words
    df = pd.DataFrame(dataframe_rows)
    return df

In [93]:
# Execute creation of dataset
himym_df = load_himym_dataset()
himym_df.head()
himym_df.count()

100%|██████████████████████████████████████████████████████████████████████████████| 139/139 [00:00<00:00, 2481.39it/s]


episode    39284
line       39284
dtype: int64

In [88]:
def process_himym_dataset(df):
    df = df[~df['line'].str.startswith("[")]
    df = df[~df['line'].str.startswith("(")]
    df['line'] = df['line'].str.strip()
    df['line'] = df['line'].str.replace(r"\(.*\)","")
    df[['character', 'line']] = df['line'].str.split(":", 1, expand=True)
    df = df[~df['line'].isnull()]
    df = df.reset_index(drop=True)
    return df
    
himym_df = process_himym_dataset(himym_df)

In [90]:
himym_df.head(20)
himym_df.count()

episode      30326
line         30326
character    30326
dtype: int64

In [46]:
# NOTE: May consider feeding one sentence and one Barney reply or multiple sentences encoded with one Barney reply
def get_barney(himym_df, level=2):
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-46-3c2367728fee>, line 1)

In [ ]:
'''
Marshall: Ciao
Ted : Sono triste!
Barney: Donne!
    
1 Sono triste!  Donne!
2 Ciao          Donne!
'''